In [5]:
import os
import openai


os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]


## ConversationBufferWindowMemory

In [6]:
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.llms import OpenAIChat
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import (
    ConversationBufferWindowMemory,
    ConversationSummaryBufferMemory,
    ConversationKGMemory,
)

template = """以下は、全知全能の全てを知っているAIのあなたとHumanが会話しています。

{history}
Human: {input}
AI:"""


def base_charactor() -> str:
    return """あなたは、親切で、創造的で、賢く、とてもフレンドリーで役に立つアシスタントです。
AIは質問に対する答えを知らない場合、正直に「知らない」と答えます。"""


prompt = PromptTemplate(input_variables=["history", "input"], template=template)

llm = OpenAIChat(
    model_name="gpt-3.5-turbo",
    temperature=0,
    max_tokens=2000,
    prefix_messages=[{"role": "system", "content": base_charactor()}],
)


conversation = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=ConversationBufferWindowMemory(k=4, memory_key="history"),
    verbose=True,
)


/Users/kamibayashi/.anyenv/envs/pyenv/versions/anaconda3-2022.10/envs/my_ai/lib/python3.10/site-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [7]:
conversation.predict(input="次の日本語を英語にしてください。'吾輩は猫である'")



> Entering new ConversationChain chain...
Prompt after formatting:
以下は、全知全能の全てを知っているAIのあなたとHumanが会話しています。


Human: 次の日本語を英語にしてください。'吾輩は猫である'
AI:

> Finished chain.


"The English translation of '吾輩は猫である' is 'I am a cat.'"

In [8]:
conversation.predict(input="フランス語だとどうなる？")



> Entering new ConversationChain chain...
Prompt after formatting:
以下は、全知全能の全てを知っているAIのあなたとHumanが会話しています。

Human: 次の日本語を英語にしてください。'吾輩は猫である'
AI: The English translation of '吾輩は猫である' is 'I am a cat.'
Human: フランス語だとどうなる？
AI:

> Finished chain.


"'吾輩は猫である'のフランス語訳はありません。ただし、フランス語で「私は猫です」と言いたい場合は、「Je suis un chat」と言います。"

In [9]:
print(conversation.memory)
print(conversation.memory.buffer)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content="次の日本語を英語にしてください。'吾輩は猫である'", additional_kwargs={}), AIMessage(content="The English translation of '吾輩は猫である' is 'I am a cat.'", additional_kwargs={}), HumanMessage(content='フランス語だとどうなる？', additional_kwargs={}), AIMessage(content="'吾輩は猫である'のフランス語訳はありません。ただし、フランス語で「私は猫です」と言いたい場合は、「Je suis un chat」と言います。", additional_kwargs={})]) output_key=None input_key=None return_messages=False human_prefix='Human' ai_prefix='AI' memory_key='history' k=4
[HumanMessage(content="次の日本語を英語にしてください。'吾輩は猫である'", additional_kwargs={}), AIMessage(content="The English translation of '吾輩は猫である' is 'I am a cat.'", additional_kwargs={}), HumanMessage(content='フランス語だとどうなる？', additional_kwargs={}), AIMessage(content="'吾輩は猫である'のフランス語訳はありません。ただし、フランス語で「私は猫です」と言いたい場合は、「Je suis un chat」と言います。", additional_kwargs={})]


In [10]:
conversation.predict(input="タイ語だとどうですか？")



> Entering new ConversationChain chain...
Prompt after formatting:
以下は、全知全能の全てを知っているAIのあなたとHumanが会話しています。

Human: 次の日本語を英語にしてください。'吾輩は猫である'
AI: The English translation of '吾輩は猫である' is 'I am a cat.'
Human: フランス語だとどうなる？
AI: '吾輩は猫である'のフランス語訳はありません。ただし、フランス語で「私は猫です」と言いたい場合は、「Je suis un chat」と言います。
Human: タイ語だとどうですか？
AI:

> Finished chain.


"'吾輩は猫である'のタイ語訳はありません。ただし、タイ語で「私は猫です」と言いたい場合は、「ผมเป็นแมว」または「ฉันเป็นแมว」と言います。"

In [11]:
conversation.predict(input="今まで私達はなにを会話しましたか？")



> Entering new ConversationChain chain...
Prompt after formatting:
以下は、全知全能の全てを知っているAIのあなたとHumanが会話しています。

Human: 次の日本語を英語にしてください。'吾輩は猫である'
AI: The English translation of '吾輩は猫である' is 'I am a cat.'
Human: フランス語だとどうなる？
AI: '吾輩は猫である'のフランス語訳はありません。ただし、フランス語で「私は猫です」と言いたい場合は、「Je suis un chat」と言います。
Human: タイ語だとどうですか？
AI: '吾輩は猫である'のタイ語訳はありません。ただし、タイ語で「私は猫です」と言いたい場合は、「ผมเป็นแมว」または「ฉันเป็นแมว」と言います。
Human: 今まで私達はなにを会話しましたか？
AI:

> Finished chain.


"私たちは、'吾輩は猫である'の英語、フランス語、タイ語への翻訳について話しました。"

## ConversationSummaryBufferMemory

In [12]:
summary_template = """会話内容を順次要約し、前回の要約に追加して新たな要約を返してください。

### 現在の要約

{summary}

### 新しい会話

{new_lines}

### 新しい要約

"""

conversation_template = """以下は、全知全能の全てを知っているAIのあなたとHumanが会話しています。
AIは質問に対する答えを知らない場合、正直に「知らない」と答えます。

{history}
Human: {input}
AI:"""

reply_llm = OpenAIChat(
    model_name="gpt-3.5-turbo",
    temperature=0.5,
    prefix_messages=[{"role": "system", "content": base_charactor()}],
)
summary_llm = OpenAIChat(model_name="gpt-3.5-turbo", temperature=0.5)

conversation_prompt = PromptTemplate(
    input_variables=["history", "input"], template=conversation_template
)
summary_prompt = PromptTemplate(
    input_variables=["summary", "new_lines"], template=summary_template
)

memory = ConversationSummaryBufferMemory(
    llm=summary_llm, max_token_limit=1000, prompt=summary_prompt
)
conversation = ConversationChain(
    llm=reply_llm, prompt=conversation_prompt, memory=memory, verbose=False
)


In [13]:
conversation.predict(input="私の名前はkamibayashiです。エンジニアをしています。")

'こんにちは、kamibayashiさん。エンジニアとしてどのような分野に携わっているのでしょうか？'

In [14]:
conversation.predict(input="私は身長175cmで体重は60kgです。")

'ごめんなさい、kamibayashiさん。私は身体的な情報を持っていないため、その情報についてはお答えできません。何か他にお力になれることはありますか？'

In [15]:
conversation.predict(input="私の特徴を要約してください。")

'はい、kamibayashiさんはエンジニアで、身長175cm、体重60kgです。その他の特徴については私には分かりません。何か他にお力になれることはありますか？'

In [16]:
conversation.predict(input='私の職業はなんですか？')

'kamibayashiさんはエンジニアとおっしゃいましたので、職業はエンジニアですね。何か他にお力になれることはありますか？'

In [17]:
conversation.predict(input="私の体重は？")

'kamibayashiさんは先程おっしゃった通り、体重は60kgですね。何か他にお力になれることはありますか？'

In [18]:
conversation.predict(input="私の名前はなんですか？")

'kamibayashiさんとおっしゃいましたね。あなたの名前はkamibayashiさんです。何か他にお力になれることはありますか？'

In [19]:
conversation.predict(input="いままでの会話を要約してください。")

'はい、kamibayashiさんはエンジニアで、身長175cm、体重60kgです。私は身体的な情報を持っていないため、その情報についてはお答えできませんでした。また、kamibayashiさんの名前はkamibayashiさんです。何か他にお力になれることはありますか？'

## Entity Memory

In [20]:
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.llms import OpenAIChat
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationEntityMemory
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

template = """あなたは全知全能の全てを知っているAIでHumanと会話しています。

{history}
Human: {input}
AI:"""


prompt = PromptTemplate(input_variables=["history", "input"], template=template)

llm = OpenAIChat(
    model_name="gpt-3.5-turbo",
    temperature=0.5,
    prefix_messages=[{"role": "system", "content": base_charactor()}],
)

conversation = ConversationChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm, k=5),
    verbose=True,
)


In [21]:
conversation.predict(input="Aくんの身長170cmで体重は60kgで職業はエンジニアです。")
conversation.predict(input="Bくんの身長は180cmで体重は70kgで職業は経営者です。")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'ありがとうございます。Bくんは身長が180cmで体重が70kgで、経営者という職業をお持ちですね。何か他にお聞きになりたいことはありますか？'

In [22]:
from pprint import pprint
pprint(conversation.memory.store)

{'Bくん': 'Bくんは身長が180cmで体重が70kgで、経営者という職業をお持ちです。',
 '経営者': 'Bくんは身長が180cmで体重が70kgで、経営者という職業をお持ちです。'}


In [23]:
conversation.predict(input="Aくんと、Bくんの特徴を説明してください。")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'Aさんはエンジニアで、身長が170cm、体重が60kgです。Bさんは経営者で、身長が180cm、体重が70kgです。また、Bさんは職業柄、経営的な視点やリーダーシップに優れている可能性が高いです。一方で、Aさんは技術的な知識やスキルに優れている可能性が高いです。ただし、これらの特徴は一般化することはできず、個人差があることに注意が必要です。'